In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# やりたきこと
* さまざまな入力センサー値（時系列など）を介して都市の大気汚染を予測する

In [ ]:
# ライブラリのロード
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
import optuna.integration.lightgbm as opt_lgb

import sklearn.model_selection

from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_squared_log_error 

from matplotlib import pyplot as plt
from itertools import product
import seaborn as sns
import io

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')

# データの中身
### 説明変数
* date_time　:　日付 
* deg_C　：　摂氏℃
* relative_humidity　：　相対湿度 
* absolute_humidity　：　絶対湿度 
* sensor_1〜５　：　センサーの値 

### 目的変数
* target_carbon_monoxide　:　一酸化炭素
* target_benzene　:　ベンゼン
* target_nitrogen_oxides : 窒素酸化物


In [ ]:
sample_submission.count()

In [ ]:
print(sample_submission)

In [ ]:
sample_submission.info()

In [ ]:
test.head()

In [ ]:
train.head()

# 分析
## 統計量確認

In [ ]:
test.describe().T

In [ ]:
train.describe().T

# 欠損値確認

In [ ]:
test.info()

In [ ]:
train.info()

→データに欠損なし

## 可視化
### 一旦とにかくヒストグラム化してみる

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10 
train.hist()
plt.tight_layout()
plt.show() 

### ヒートマップで相関関係を可視化

In [ ]:
# trainとtestをunion
train.loc[:,'test'] = 0
test.loc[:,'test'] = 1

df = train.append(test, ignore_index=False)

#### ymdh分解

In [ ]:
df.loc[:,'year'] = pd.to_datetime(df['date_time']).dt.year
df.loc[:,'month'] = pd.to_datetime(df['date_time']).dt.month
df.loc[:,'day'] = pd.to_datetime(df['date_time']).dt.day
df.loc[:,'hour'] = pd.to_datetime(df['date_time']).dt.hour
df.loc[:,'dayofweek'] = pd.to_datetime(df['date_time']).dt.dayofweek
df.loc[:,'is_weekend'] = (df["dayofweek"] >= 5).astype("int")

In [ ]:
#df.loc[:,'is_go_to_work'] = df['hour'].isin(np.arange(7, 9, 1)).astype("int")
#df.loc[:,'is_go_home'] = df['hour'].isin(np.arange(18, 20, 1)).astype("int")
df.loc[:,'is_work_hour'] = df['hour'].isin(np.arange(8, 21, 1)).astype("int")

In [ ]:
df.loc[:,'is_winter'] = df['month'].isin([1, 2, 12])
df.loc[:,'is_spring'] = df['month'].isin([3, 4, 5])
df.loc[:,'is_summer'] = df['month'].isin([6, 7, 8])
df.loc[:,'is_autumn'] = df['month'].isin([9, 10, 11])

In [ ]:
df.loc[:, 'SMC'] = (df['absolute_humidity'] * 100) / df['relative_humidity']
df.loc[:, 'hm_1'] = ((df['absolute_humidity'].values <= 0.25)).astype(int)

#### 正規化

In [ ]:
#cols = ['deg_C','relative_humidity','absolute_humidity','sensor_1','sensor_2','sensor_3','sensor_4','sensor_5']
#for col in cols:
#    df.loc[:, col] = preprocessing.minmax_scale(df[col])

#### lag 特徴量の生成

In [ ]:
lag_features = ['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']
lags = [3, 6, 9, 12, 24]
for feature in lag_features:
    for lag in lags:
        df[feature + '_lag_' + str(lag)] = df[feature] - df[feature].shift(periods=lag, fill_value=0)

In [ ]:
df.describe().T

In [ ]:
heatmap = df[df['test'] == 0].drop(columns='test')
heatmap.corr()

In [ ]:
corr = heatmap.corr()
sns.set_context("notebook", font_scale=1.0, rc={"lines.linewidth": 2.5})
plt.figure(figsize=(36,18))
a = sns.heatmap(corr, annot=True, fmt='.2f')
rotx = a.set_xticklabels(a.get_xticklabels(), rotation=90)
roty = a.set_yticklabels(a.get_yticklabels(), rotation=30)

* 時間と月は特徴量として効きそう。年日は微妙
* target_carbon_monoxide と他のtaergetは相関関係が強い
* target_benzene > target_carbon_monoxide > target_nitrogen_oxides の順にpredする？
 * target_benzeneは、targetを全部drop
 * target_carbon_monoxideは、target_benzene以外をdrop
 * target_nitrogen_oxidesは、target_nitrogen_oxidesをdrop


## データセットの作成

In [ ]:
# 不要カラム削除
df = df.drop(columns=['year','day'])

# 学習データを抽出
tmp_df = df[df['test'] == 0]
tmp_df = tmp_df.drop(columns='date_time')
tmp_df = tmp_df.drop(columns='test')

# データの30%をテストに、70%を学習に利用する
train_df, valid_df = sklearn.model_selection.train_test_split(tmp_df, test_size=0.3, train_size = 0.7, random_state = 0)

b_train_df = train_df.drop(columns=['target_carbon_monoxide','target_nitrogen_oxides'])
b_valid_df = valid_df.drop(columns=['target_carbon_monoxide','target_nitrogen_oxides'])

co_train_df = train_df.drop(columns='target_nitrogen_oxides')
co_valid_df = valid_df.drop(columns='target_nitrogen_oxides')

no_valid_df = valid_df.drop(columns='target_benzene')
no_train_df = train_df.drop(columns='target_benzene')


In [ ]:
# pred 用データセット
pred_df = df[df['test'] == 1 ]
pred_df = pred_df.drop(columns=['target_carbon_monoxide','target_benzene','target_nitrogen_oxides'])
pred_df = pred_df.drop(columns='test')

In [ ]:
pred_df.info()

In [ ]:
b_train_df.info()

# RMSLE

In [ ]:
def rmsle_lgbm(y_pred, data):
    y_true = np.array(data.get_label())
    #msle = mean_squared_log_error(y_true, y_pred)
    rmsle = np.sqrt(np.mean(np.power(np.log1p(y_true + 1) - np.log1p(y_pred + 1), 2)))
    #rmsle = np.sqrt(msle)
    return 'rmsle', rmsle, False

# 学習 target_carbon_monoxide

In [ ]:
params = {
 'task': 'train',
 'boosting_type': 'gbdt',    
 'objective': 'regression',
 'metric': 'rmse',
 'learning_rate': 0.05,

 'feature_pre_filter': False,
 'num_leaves': 16,
 'feature_fraction': 0.5,
 'bagging_fraction': 0.9321963910217531,
 'bagging_freq': 4,
 'min_child_samples': 20,

 'num_iterations': 20000,
 'early_stopping_round': 100,
 'verbosity': -1
}

In [ ]:
train_ds = lgb.Dataset(b_train_df.drop(columns='target_benzene'), label=b_train_df['target_benzene'])
valid_ds = lgb.Dataset(b_valid_df.drop(columns='target_benzene'), label=b_valid_df['target_benzene'])

In [ ]:
# ハイパラチューニング
# model_o = opt_lgb.train(
#     params,
#     train_ds,
#     valid_names=['train', 'valid'],
#     valid_sets=[train_ds, valid_ds],
#     verbose_eval=100,
#     early_stopping_rounds=100,
#     feval=rmsle_lgbm
# )

In [ ]:
# model_o.best_score

In [ ]:
#prms = model_o.params
#prms['metric'] = 'custom'
#print(prms)

In [ ]:
prms = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'custom',
    'learning_rate': 0.03,
    'feature_pre_filter': False,
    'num_leaves': 43,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.9321963910217531,
    'bagging_freq': 4,
    'verbosity': -1,
    'min_data_in_leaf': 20,
    'lambda_l1': 0.00024983046095551723,
    'lambda_l2': 4.532572245829988e-05,
    'min_child_samples': 20,
    'num_iterations': 20000,
    'early_stopping_round': 100
}

In [ ]:
model = lgb.train(
    prms,
    train_ds,
    valid_names=['train', 'valid'],
    valid_sets=[train_ds, valid_ds],
    verbose_eval=100,
    feval=rmsle_lgbm
)

In [ ]:
# 特徴量の重要度をプロットする
lgb.plot_importance(model, importance_type='gain')
plt.show()

In [ ]:
lgb.plot_importance(model, importance_type='split')
plt.show()

In [ ]:
tmp_pred = pred_df
val_pred = model.predict(tmp_pred.drop(columns='date_time'))

In [ ]:
print(val_pred)

In [ ]:
result_tmp = pred_df
result_tmp['target_benzene'] = val_pred

In [ ]:
train_ds = lgb.Dataset(co_train_df.drop(columns='target_carbon_monoxide'), label=co_train_df['target_carbon_monoxide'])
valid_ds = lgb.Dataset(co_valid_df.drop(columns='target_carbon_monoxide'), label=co_valid_df['target_carbon_monoxide'])

In [ ]:
# ハイパラチューニング
#model_o = opt_lgb.train(
#    params,
#    train_ds,
#    valid_names=['train', 'valid'],
#    valid_sets=[train_ds, valid_ds],
#    verbose_eval=100,
#    early_stopping_rounds=100,
#    feval=rmsle_lgbm
#)

In [ ]:
# model_o.best_score

In [ ]:
# prms = model_o.params
# prms['metric'] = 'custom'
# print(prms)

In [ ]:
prms = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'custom',
    'learning_rate': 0.005,
    'feature_pre_filter': False,
    'num_leaves': 32,
    'feature_fraction': 0.484,
    'bagging_fraction': 0.7728239157212622,
    'bagging_freq': 4,
    'verbosity': -1,
    'min_data_in_leaf': 20,
    'lambda_l1': 0.14473721180738666,
    'lambda_l2': 1.4136380974203801e-08,
    'min_child_samples': 20,
    'num_iterations': 20000,
    'early_stopping_round': 100
}

In [ ]:
model = lgb.train(
    prms,
    train_ds,
    valid_names=['train', 'valid'],
    valid_sets=[train_ds, valid_ds],
    verbose_eval=100,
    feval=rmsle_lgbm
)

In [ ]:
# 特徴量の重要度をプロットする
lgb.plot_importance(model, importance_type='gain')
plt.show()

In [ ]:
lgb.plot_importance(model, importance_type='split')
plt.show()

In [ ]:
pred_df = result_tmp
tmp_pred = pred_df
val_pred = model.predict(tmp_pred.drop(columns='date_time'))

In [ ]:
result_tmp = pred_df
result_tmp['target_carbon_monoxide'] = val_pred

In [ ]:
train_ds = lgb.Dataset(no_train_df.drop(columns='target_nitrogen_oxides'), label=no_train_df['target_nitrogen_oxides'])
valid_ds = lgb.Dataset(no_valid_df.drop(columns='target_nitrogen_oxides'), label=no_valid_df['target_nitrogen_oxides'])

In [ ]:
# ハイパラチューニング
#model_o = opt_lgb.train(
#    params,
#    train_ds,
#    valid_names=['train', 'valid'],
#    valid_sets=[train_ds, valid_ds],
#    verbose_eval=100,
#    early_stopping_rounds=100,
#    feval=rmsle_lgbm
#)

In [ ]:
#model_o.best_score

In [ ]:
#prms = model_o.params
#prms['metric'] = 'custom'
#print(prms)

In [ ]:
prms = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'custom',
    'learning_rate': 0.02,
    'feature_pre_filter': False,
    'num_leaves': 55,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.6261980989290583,
    'bagging_freq': 1,
    'verbosity': -1,
    'min_data_in_leaf': 20,
    'lambda_l1': 0.0022756588557693758,
    'lambda_l2': 0.44304416643833433,
    'min_child_samples': 20,
    'num_iterations': 20000,
    'early_stopping_round': 100
}

In [ ]:
model = lgb.train(
    prms,
    train_ds,
    valid_names=['train', 'valid'],
    valid_sets=[train_ds, valid_ds],
    verbose_eval=100,
    feval=rmsle_lgbm
)

In [ ]:
# 特徴量の重要度をプロットする
lgb.plot_importance(model, importance_type='gain')
plt.show()

In [ ]:
lgb.plot_importance(model, importance_type='split')
plt.show()

In [ ]:
pred_df = result_tmp
tmp_pred = tmp_pred.drop(columns='target_benzene')

val_pred = model.predict(tmp_pred.drop(columns='date_time'))

In [ ]:
result_tmp = pred_df
result_tmp['target_nitrogen_oxides'] = val_pred

In [ ]:
result = result_tmp.loc[:,['date_time','target_carbon_monoxide','target_benzene','target_nitrogen_oxides']]

In [ ]:
result.head()

In [ ]:
result.to_csv('submission.csv', index=False)